In [1]:
!date
!python -V # check if python version is later than 3.8

Tue Apr 18 15:14:44 JST 2023
Python 3.9.5


In [2]:
pip install msticpy

You should consider upgrading via the '/Users/hacket/.pyenv/versions/3.9.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import msticpy as mp
print(mp.__version__)

2.4.0


In [5]:
mp.init_notebook()

ModuleNotFoundError: No module named 'azure.mgmt.resource'

In [6]:
!pip install azure-mgmt-resource
!pip install azure-identity
!pip install msticpy\[azure]

  Using cached azure_mgmt_resource-23.0.0-py3-none-any.whl (2.5 MB)
You should consider upgrading via the '/Users/hacket/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/hacket/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.
  Using cached keyring-23.13.1-py3-none-any.whl (37 kB)
  Using cached azure_mgmt_monitor-6.0.0-py3-none-any.whl (1.3 MB)
  Using cached azure_mgmt_network-23.0.0-py3-none-any.whl (642 kB)
  Using cached azure_keyvault_secrets-4.7.0-py3-none-any.whl (348 kB)
  Using cached azure_mgmt_compute-29.1.0-py3-none-any.whl (7.2 MB)
  Using cached azure_mgmt_resourcegraph-8.0.0-py2.py3-none-any.whl (27 kB)
  Using cached azure_mgmt_keyvault-10.2.1-py3-none-any.whl (779 kB)
  Using cached azure_storage_blob-12.16.0-py3-none-any.whl (387 kB)
  Using cached jaraco.classes-3.2.3-py3-none-any.whl (6.0 kB)
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
You should consi

In [9]:
!pip install msticpy\[splunk]


  Using cached splunk_sdk-1.7.3-py3-none-any.whl
You should consider upgrading via the '/Users/hacket/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import msticpy as mp
mp.init_notebook()

2023-04-18 15:20:19,840: WARNING - config validation error Missing or empty 'AzureSentinel' section (nbinit#697)
2023-04-18 15:20:19,841: WARNING - Could not find msticpyconfig.yaml in standard search. (nbinit#710)


In [2]:
splunk_prov = mp.QueryProvider("Splunk")
splunk_prov.connect()

connected


In [3]:
splunk_prov.SplunkGeneral.get_events_parameterized('?')

Query:  get_events_parameterized
Data source:  Splunk
Generic parameterized query from index/source

Parameters
----------
add_query_items: str (optional)
    Additional query clauses
    (default value is: | head 100)
end: datetime
    Query end time
index: str (optional)
    Splunk index name
    (default value is: *)
project_fields: str (optional)
    Project Field names
    (default value is: | table TimeCreated, host, EventID, EventDescripti...)
source: str (optional)
    Splunk source type
    (default value is: *)
start: datetime
    Query start time
timeformat: str (optional)
    Datetime format to use in Splunk query
    (default value is: "%Y-%m-%d %H:%M:%S.%6N")
Query:
 search index={index} source={source} timeformat={timeformat} earliest={start} latest={end} {project_fields} {add_query_items}


In [4]:
splunk_prov.SplunkGeneral.get_events_parameterized('print',
    index="botsv2",
    source="WinEventLog:Microsoft-Windows-Sysmon/Operational",
    timeformat='"%Y-%m-%d %H:%M:%S"',
    start="2017-08-25 00:00:00",
    end="2017-08-25 10:00:00"
)

' search index=botsv2 source=WinEventLog:Microsoft-Windows-Sysmon/Operational timeformat="%Y-%m-%d %H:%M:%S" earliest="2017-08-25 00:00:00" latest="2017-08-25 10:00:00" | table TimeCreated, host, EventID, EventDescription, User, process, cmdline, Image, parent_process, ParentCommandLine, dest, Hashes | head 100'

In [5]:
default_df = splunk_prov.SplunkGeneral.get_events_parameterized(
    index="botsv2",
    source="WinEventLog:Microsoft-Windows-Sysmon/Operational",
    start="2017-08-25 00:00:00.000000",
    end="2017-08-25 10:00:00.000000"
)
len(default_df) # 100 because of add_query_items = '| head 100' by default

Waiting Splunk job to complete: 200.0it [00:03, 65.68it/s]                         


100

In [6]:
splunk_prov.SplunkGeneral.get_events_parameterized('print',
    index="botsv2",
    source="WinEventLog:Microsoft-Windows-Sysmon/Operational",
    start="2017-08-25 00:00:00.000000",
    end="2017-08-25 10:00:00.000000",
    add_query_items=''
)

' search index=botsv2 source=WinEventLog:Microsoft-Windows-Sysmon/Operational timeformat="%Y-%m-%d %H:%M:%S.%6N" earliest="2017-08-25 00:00:00" latest="2017-08-25 10:00:00" | table TimeCreated, host, EventID, EventDescription, User, process, cmdline, Image, parent_process, ParentCommandLine, dest, Hashes '

In [7]:
unlimited_df = splunk_prov.SplunkGeneral.get_events_parameterized(
    index="botsv2",
    source="WinEventLog:Microsoft-Windows-Sysmon/Operational",
    start="2017-08-25 00:00:00.000000",
    end="2017-08-25 10:00:00.000000",
    add_query_items='',
    count=0
)
len(unlimited_df)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:02<00:00, 48.91it/s]
/Users/hacket/.pyenv/versions/3.9.5/lib/python3.9/site-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


100

### => msticpy 2.4.0 has a bug to fetch full records from splunk !!!

In [8]:
# check total record counts of above query
splunk_prov.SplunkGeneral.get_events_parameterized(
    index="botsv2",
    source="WinEventLog:Microsoft-Windows-Sysmon/Operational",
    start="2017-08-25 00:00:00.000000",
    end="2017-08-25 10:00:00.000000",
    add_query_items='| stats count',
    count=0
)


Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 98.21it/s]
/Users/hacket/.pyenv/versions/3.9.5/lib/python3.9/site-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,count
0,11268


The data store's record counts = 11268 expected !